# Research Track Assignment 2

In [2]:
%matplotlib tk

import sys
import os
import rospy
import actionlib
import actionlib.msg
from nav_msgs.msg import Odometry
from geometry_msgs.msg import PoseStamped
from robot_sim.msg import OdoSensor
from sensor_msgs.msg import LaserScan
from robot_sim.msg import PlanningAction, PlanningGoal, PlanningActionFeedback
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FuncAnimation
from IPython.display import display
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, Label, Text 

goal_x = 0.0
goal_y = 0.0
robot_x = None
robot_y = None
goals_reached = 0
goals_cancelled = 0

fig = None
axs = None
client = None
gui = None
counter = 0

update_rate = 100
draw_rate = 10 * update_rate

colors = ["green", "orange", "red"]

regions_ = {
    'right': 0.0,
    'fright': 0.0,
    'front': 0.0,
    'fleft': 0.0,
    'left': 0.0,
}

odoMsg = OdoSensor()

# UI Code

In [3]:
def calculate_color(distance):
    global colors
    i = 0
    if distance >3.0:
        i = 0
    if (distance>1.0 and distance<=3.0):
        i = 1
    if distance <=1.0:
        i = 2
    return colors[i]

def find_minimum_distance():
    global regions_
    
    min_distance = 100000
    for key in regions_.keys():
        if regions_[key] < min_distance:
            min_distance = regions_[key]
    return min_distance
    
send_goal = Button(description ='Set Goal')
send_goal.style = ButtonStyle(button_color='green')
send_goal.layout = Layout(width='auto', height="20%", margin="20px")
             
x_goal = Text()
x_goal_container = HBox([Label(value='Xg'), x_goal ], layout=Layout(display='flex', flex_flow='row', justify_content='space-between'))

y_goal = Text()
y_goal_container = HBox([Label(value='Yg'), y_goal], layout=Layout(display='flex', flex_flow='row', justify_content='space-between', state='disabled'))
        
obstacle_distance = Text(placeholder='Unknown', description='Min:', disabled=True)
obstacle_distance_container = HBox([obstacle_distance], layout=Layout(display='flex', flex_flow='row', justify_content='space-between', state='disabled'))
        
navigation_box_label = Label('Enter a New Goal:')
navigation_box = VBox([navigation_box_label, x_goal_container, y_goal_container, send_goal])
navigation_box.layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid', width='50%', padding="5px")
        
monitoring_box = VBox([Label('Min Obstacle Distance:'), obstacle_distance_container])
monitoring_box.layout = Layout(border='solid', width='50%',padding="10px")
        
main_widget = VBox([navigation_box, monitoring_box])
main_widget.layout = Layout(width='100%', height='100%')

class OdoVisualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.fig.set_figwidth(8)
        self.fig.set_figheight(6)
        self.ln, = plt.plot([], [], marker = 'o', color='g')
        self.x_data, self.y_data = [] , []
        
    def plot_init(self):
        self.ax.set_title("Robot - Goal Positions")
        self.ax.set_xlim(-15, 15)
        self.ax.set_ylim(-15, 15)
        return self.ln
    
    def odom_callback(self, x, y):
        self.x_data.append(x)
        self.y_data.append(y)
    
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

class GoalVisualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.fig.set_figwidth(8)
        self.fig.set_figheight(6)
        self.goals_status = ['Reached', 'Cancelled']
        self.gaols_value = [0, 0]
       
    def plot_init(self):
        self.ax.set_title("Reached-Cancelled Goals")

    def goal_callback(self, reached, cancelled):
        self.gaols_value = [reached, cancelled]
        
    def update_plot(self, frame):
        self.ax.cla()
        self.ax.bar(self.goals_status, self.gaols_value, color=['blue', 'red'], width = 1)
        
vis2 = GoalVisualiser()
ani2 = FuncAnimation(vis2.fig, vis2.update_plot, init_func=vis2.plot_init, interval=draw_rate)

vis1 = OdoVisualiser()
ani1 = FuncAnimation(vis1.fig, vis1.update_plot, init_func=vis1.plot_init, interval=draw_rate)

def updateGUI():
    global robot_x, robot_y, vis
    obstacle_distance.value = str(find_minimum_distance())
    vis1.odom_callback(robot_x, robot_y)
    
def handle_send_goal_click(event):
    global goal_x, goal_y, client
    goal_x = x_goal.value
    goal_y = y_goal.value
    circle1 = plt.Circle((float(goal_x), float(goal_y)), 0.6, color='black')
    vis1.ax.add_patch(circle1)
    sendGoal(client, float(goal_x), float(goal_y))
    
send_goal.on_click(handle_send_goal_click) 


display(main_widget)
updateGUI()

# ROS Code

In [4]:
def createClient():
    try:
        client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
    except rospy.ROSInterruptException:
        print("Could not create the client", file=sys.stderr) 
    return client

def sendGoal(client, x, y):
    target_pose = PoseStamped()
    
    try:
        client.wait_for_server()
        
        target_pose.pose.position.x = x
        target_pose.pose.position.y = y       
        goal = PlanningGoal(target_pose)

        client.send_goal(goal)
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)
        
def setOdoMessage(msg):
    global robot_x, robot_y, counter, trajectory_x, trajectory_y
    robot_x = msg.pose.pose.position.x
    robot_y = msg.pose.pose.position.y
 
    counter +=1 
    if counter == update_rate:
        updateGUI()
        counter = 0

def clbk_laser(msg):
    global regions_, counter
    regions_ = {
        'right':  min(min(msg.ranges[0:143]), 10),
        'fright': min(min(msg.ranges[144:287]), 10),
        'front':  min(min(msg.ranges[288:431]), 10),
        'fleft':  min(min(msg.ranges[432:575]), 10),
        'left':   min(min(msg.ranges[576:713]), 10),
    }
    if counter == update_rate:
        updateGUI()
        
def updateGoalSummary(msg):
    global goals_reached
    global goals_cancelled

    state = msg.feedback.stat
    if state == "Target reached!":
        goals_reached = goals_reached + 1

    if state == "Target cancelled!":
        goals_cancelled = goals_cancelled + 1  
        
    vis2.goal_callback(goals_reached, goals_cancelled)


def main():
    global client
    
    rospy.init_node('notebook_node')
    client = createClient()
    odoMessageSubscriber = rospy.Subscriber("odom", Odometry, setOdoMessage)
    goalSummarySubscriber = rospy.Subscriber("reaching_goal/feedback", PlanningActionFeedback, updateGoalSummary)
    sub = rospy.Subscriber('/scan', LaserScan, clbk_laser)
    
if __name__ == "__main__":
    main()
